In [1]:
# Скачивание документов из папки articles
import glob


articles_pdf = glob.glob('./articles/*.pdf')
articles_docx = glob.glob('./articles/*.doc*')

In [2]:
articles_docx

['./articles/«Инициатива по саморегулированию в индустрии генеративного ИИ».docx',
 './articles/Состояние индустрии больших ИИ-моделей в Китае в 2024 г.doc']

In [3]:
articles_text = []

In [4]:
# Выделение текста из pdf файлов
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                              chunk_overlap=100)

for art_pdf in articles_pdf:
    loader = PyPDFLoader(art_pdf)
    for chunk_art_pdf in loader.load_and_split(text_splitter):
        articles_text.append(chunk_art_pdf)

In [5]:
# Выделение текста из docx файлов
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
for art_docx in articles_docx:
    loader = UnstructuredWordDocumentLoader(art_docx)
    for chunk_art_docx in loader.load_and_split(text_splitter):
        articles_text.append(chunk_art_docx)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = Chroma.from_documents(articles_text, embedding=embedding)

/usr/lib/python3.10/importlib/__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
/tmp/ipykernel_16554/1436424486.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langch

## Запуск на LLama 3

In [ ]:
import requests
import json


def client(question):
    url = "http://192.168.1.152:8888/api/echo"
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        'prompt' : question
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()['answer']

In [28]:
question = 'What factors contribute to the success of Russian teams in international competitions in math, programming, and science, and how does the presence of a competitive AI community in Russia affect these achievements?'

context = vector_store.similarity_search(
    question,
    k=2
)

context_text = ['\n'.join([i.page_content for i in context])]


prompt = f'''Ты ассистент, который очень хочет помочь.\
Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context_text} \
Вопрос: {question} \
Ответ:'''
import requests
import json

url = "http://192.168.1.152:8888/api/echo"
headers = {
    "Content-Type": "application/json"
}
data = {
    'prompt' : prompt
}

response = requests.post(url, headers=headers, data=json.dumps(data))
response.json()['answer']

TypeError: string indices must be integers

In [29]:
type(response)

requests.models.Response

In [33]:
print(response.json()['answer'])

Судя по предоставленной информации, факторы, которые способствуют успеху российских команд в международных конкурсах по математике, программированию и науке, следующие:

1. Регулярные победы российских команд на школьных и студенческих олимпиадах по математике, информатике и программированию.
2. Вход России в десятку стран-лидеров по количеству научных публикаций по физике, математике и химии.

Присутствие активного и растущего сообщества специалистов по обработке данных с использованием искусственного интеллекта в Российской Федерации также является важным фактором. Это сообщество регулярно представляет результаты своей работы на научных конференциях мирового уровня, что способствует развитию исследований в этой области.

В частности, более 50% российских ученых, работающих в исследовательских центрах, публикуют свои работы на конференциях по искусственному интеллекту уровня А*, а более 30% организаций, осуществляющих деятельность в различных отраслях экономики, применяют технологии и